<a href="https://colab.research.google.com/github/AbbhinavJayaraman/HCI-Grad-Course/blob/main/homegrown_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Since ChatGPT and existing LLMs are being annoying, we are just going to follow an existing setup that can be tweaked, making use of ollama since performance on one of our laptops running an modified Fedora Linux distrubtion was able tobe fast and responsive.

Still need to figure out how to connect this with either the PRAAT scripts or myprosody:

Links:
Whisper + ollama + Bark framework: https://medium.com/@vndee.huynh/build-your-own-voice-assistant-and-run-it-locally-whisper-ollama-bark-c80e6f815cba

PRAAT in python: https://github.com/YannickJadoul/Parselmouth
myprosody: https://github.com/Shahabks/myprosody


For all the following packages to install properly, you will need to make sure you on python version 3.12

Install python version 3.12 (we used Homebrew), and then made a python venv with it (python3.12 -m venv sts), where sts is just a name of the venv.

then you can activate the venv using source sts/bin/activate

also make sure you have the portaudio library installed. For Mac, make use of the Homebrew package manager. 

For linux distrubitions, it's best to use the default package manager's version of portaudio, as well as the development headers (for us, that was portaudio-devel on Fedora Silverblue Linux)

In [137]:
# %pip install nltk
# %pip install torch
# %pip install numpy
# %pip install transformers
# %pip install openai-whisper
# %pip install sounddevice
# %pip install rich
# %pip install langchain
# %pip install langchain_community
# %pip install scipy

In [138]:
import shutil
import os

# Save the STT file locally as generic name
# process transcription
# copy file to archive, or just return and do that after.

# get the output, can just not print also

# Function to delete a file
def delete_file(file_path):
    try:
        os.remove(file_path)
        print(f"File {file_path} deleted successfully.")
    except FileNotFoundError:
        print(f"File {file_path} not found.")
    except PermissionError:
        print(f"Permission denied: {file_path}.")
    except Exception as e:
        print(f"Error occurred while deleting file {file_path}: {e}")


def copy_file (dst: str, src: str, rm_src: bool=False) :
    try:
        shutil.copy(src, dst)  # Preserves metadata like timestamps
        print(f"File copied successfully from {src} to {dst}")
    except Exception as e:
        print(f"An error occurred: {e}")

    if rm_src :
        delete_file(src)
        



In [139]:
import myprosody as mysp
import numpy as np

import io
import sys

# Redirect the print output
def detect_sr(src: str) -> int:
    # Create a StringIO object to capture the output
    p=src
    c=r"/var/home/jayabbhi/Documents/HCI_grad_project/myprosody/myprosody"

    captured_output = io.StringIO()
    sys.stdout = captured_output  # Redirect sys.stdout to the StringIO object
    try:
        # Call the function whose output you want to capture
        mysp.myspsr(p,c)
    finally:
        sys.stdout = sys.__stdout__  # Restore the original sys.stdout

    # Get the captured output as a string
    output = captured_output.getvalue()
    captured_output.close()  # Close the StringIO object
    
    final_syl_sec = 8
    try:
        final_syl_sec = int(output.split(" ")[1].strip())
    except Exception as e:
        print(output)
    
    return final_syl_sec




Before running the program, make sure that ollama is installed (we used Homebrew). You can then use the 'ollama serve' to start Ollama up, use 'ollama pull mistral' to pull a specific LLM (mistral, in this case). 

You do have to specify the name in llm input variable to the ConversationChain.

In [140]:
import threading
import whisper
import sounddevice as sd
from queue import Queue
from rich.console import Console
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama

stt = whisper.load_model("tiny")
console = Console()

template = """
You are a helpful and friendly AI assistant. You are polite and respectful. You will only give one response, in the same language as the input text.
The conversation transcript is as follows:
{history}
And here is the user's follow-up: {input}
Your response:
"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
chain = ConversationChain(
    prompt=PROMPT,
    verbose=False,
    memory=ConversationBufferMemory(ai_prefix="Assistant:"),
    llm=Ollama(model="mistral"),
)

/var/home/jayabbhi/Documents/HCI_grad_project/lets-talk-tempo/sts/lib/python3.12/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpo

In [141]:
from scipy.io.wavfile import read
from scipy.signal import resample
from pydub import AudioSegment
import librosa
import time


def record_audio(stop_event, data_queue):
    """
    Captures audio data from the user's microphone and adds it to a queue for further processing.
    Args:
        stop_event (threading.Event): An event that, when set, signals the function to stop recording.
        data_queue (queue.Queue): A queue to which the recorded audio data will be added.
    Returns:
        None
    """
    def callback(indata, frames, time, status):
        if status:
            console.print(status)
        data_queue.put(bytes(indata))

    with sd.RawInputStream(
        samplerate=44100, dtype="int32", channels=1, callback=callback
    ):

        while not stop_event.is_set():
            time.sleep(0.1)

def transcribe(audio_np: np.ndarray) -> str:
    """
    Transcribes the given audio data using the Whisper speech recognition model.
    Args:
        audio_np (numpy.ndarray): The audio data to be transcribed.
    Returns:
        str: The transcribed text.
    """
    result = stt.transcribe(audio_np, fp16=False)  # Set fp16=True if using a GPU
    text = result["text"].strip()
    return text

def get_llm_response(text: str) -> str:
    """
    Generates a response to the given text using the Llama-2 language model.
    Args:
        text (str): The input text to be processed.
    Returns:
        str: The generated response.
    """
    response = chain.predict(input=text)
    if response.startswith("Assistant:"):
        response = response[len("Assistant:") :].strip()
    return response

# def play_audio(audio_file):
#     """
#     Plays the given audio data using the sounddevice library.
#     Args:
#         sample_rate (int): The sample rate of the audio data.
#         audio_array (numpy.ndarray): The audio data to be played.
#     Returns:
#         None
#     """

#     audio = AudioSegment.from_mp3(audio_file)

#     # Convert to numpy array (this gives us access to the raw audio data)
#     audio_data = np.array(audio.get_array_of_samples())

    
#     # Adjust the playback speed by resampling
#     # For example, to double the speed:
#     sd.play(audio_data, audio.frame_rate)
#     sd.wait()

def play_audio(audio_file, user_speech_rate):
    audio_data, sr = librosa.load(audio_file, sr=None)

    # Slow down the audio (e.g., 50% slower)
    speed_factor = float(8.0 / user_speech_rate)
    print(f'New speech rate: {speed_factor}')
    audio_stretched = librosa.effects.time_stretch(audio_data, rate=speed_factor)

    # Play the slowed-down audio
    # sd.play(audio_stretched, sr)
    sd.play(audio_stretched, sr)
    sd.wait()

In [142]:
from scipy.io import wavfile
from gtts import gTTS

sample_rate = 44100

if __name__ == "__main__":
    console.print("[cyan]Assistant started! Press Ctrl+C to exit.")

    try:
        while True:
            console.input(
                "Press Enter to start recording, then press Enter again to stop."
            )

            data_queue = Queue()  # type: ignore[var-annotated]
            stop_event = threading.Event()
            recording_thread = threading.Thread(
                target=record_audio,
                args=(stop_event, data_queue),
            )
            recording_thread.start()

            input()
            stop_event.set()
            recording_thread.join()

            audio_data = b"".join(list(data_queue.queue))
            audio_np = (np.frombuffer(audio_data, dtype=np.int32))

            if audio_np.size > 0:
                with console.status("Transcribing...", spinner="earth"):
                    # The Audio file exists here, we can pass it along to myprosody
                    wavfile.write('user_fr.wav', sample_rate, audio_np)
                    transcr = (stt.transcribe('user_fr.wav'))['text']
                # insert with method to extract the data speech rate method we need
                console.print(f"[yellow]You: {transcr}")

                with console.status("Extracting SR... ", spinner="earth"):
                    copy_file('../myprosody/myprosody/dataset/audioFiles/', 'user_fr.wav')
                    user_sr = detect_sr('user_fr')
                    print(f'User Speech Rate: {user_sr} (syllables per second)')

                with console.status("Generating response...", spinner="earth"):
                    response = get_llm_response(transcr)
                    console.print(f"[cyan]Assistant: {response}")
                    
                # Generate, save, and play the audio
                with console.status("Processing assistant audio..", spinner="earth"):
                    tts = gTTS(text=response, lang='en')
                    tts.save("gtts.mp3")
                    play_audio("gtts.mp3", user_sr)

                with console.status("Cleaninp up temp files..", spinner="earth"):
                    delete_file("user_fr.wav")
                    delete_file("gtts.mp3")
                    delete_file("../myprosody/myprosody/dataset/audioFiles/user_fr.wav")
            else:
                console.print(
                    "[red]No audio recorded. Please ensure your microphone is working."
                )

    except KeyboardInterrupt:
        console.print("\n[red]Exiting...")

    console.print("[blue]Session ended.")

Assistant started! Press Ctrl+C to exit.

Press Enter to start recording, then press Enter again to stop.

Output()

You:  What is the definition of the word erodosa claitis?

File copied successfully from user_fr.wav to ../myprosody/myprosody/dataset/audioFiles/

...

Assistant:  Apologies for any confusion. Based on my research, the term "erodosa claitis" does not appear to be a 
recognized medical or scientific term. It might be a typo or an error in the provided phrase. If you meant a 
different term, kindly let me know so I can help you better.

Output()

New speech rate: 1.0

Exiting...

Session ended.